In [1]:
import datetime as dt
import pandas as pd
import numpy as np
import os
import re
#import networkx as nx
import matplotlib.pyplot as plt
import requests
import json 
import sys
import warnings
from tqdm.notebook import tqdm
import time
import nltk
from nltk.corpus import stopwords

## 个股研报

In [175]:
df

,researcher,orgSName,orgCode,indvInduName,indvInduCode,stockName,stockCode,publishDate,title,content,url
0,郑薇,国联证券,80000028,医疗器械,1041,诺唯赞,688105,2022-02-17 00:00:00.000,从分子试剂走向生命科学，多元布局打开成长空间,诺唯赞(688105)投资要点：分子生物试剂龙头，多元化拓展保持高速成长公司立足酶、蛋白等关...,https://data.eastmoney.com/report/zw_stock.jsh...
1,"吴吉森,厉秋迪",国海证券,10001055,专用设备,910,光力科技,300480,2022-02-17 00:00:00.000,半导体划片机国产替代领头羊,光力科技(300480)投资要点切割是封装环节关键步骤，下游扩产助推划片机市场快速成长。封装...,https://data.eastmoney.com/report/zw_stock.jsh...
2,"王华君,李思扬",浙商证券,80000206,光伏设备,1031,奥特维,688516,2022-02-17 00:00:00.000,奥特维点评：获晶澳组件设备中标，订单持续向好；期待半导体设备再突破,奥特维(688516)投资要点获晶澳组件设备中标；预计2021年新签订单超35亿元、在手订单...,https://data.eastmoney.com/report/zw_stock.jsh...
3,蒋颖,信达证券,80092742,通信设备,448,中天科技,600522,2022-02-17 00:00:00.000,联手海力风电，海缆及海工一体化能力持续强化,中天科技(600522)事件：2022年2月16日，中天科技全资子公司中天科技集团海洋工程有...,https://data.eastmoney.com/report/zw_stock.jsh...
4,"李永磊,董伯骏",国海证券,10001055,化学原料,1019,华鲁恒升,600426,2022-02-17 00:00:00.000,守正出奇，新十年再出发,华鲁恒升(600426)核心观点纵观三十年历史，多元化发展战略坚定：1991年-2000年，...,https://data.eastmoney.com/report/zw_stock.jsh...
...,...,...,...,...,...,...,...,...,...,...,...
495,"马金龙,刘岗",浙商证券,80000206,能源金属,1015,天齐锂业,002466,2022-01-29 00:00:00.000,天齐锂业深度报告：扩产乘风资源价格重估，锂资源巨头王者归来,天齐锂业(002466)布局全球优质锂资源，龙头地位稳固公司通过控股泰利森、SQM等全球最优...,https://data.eastmoney.com/report/zw_stock.jsh...
496,"马天诣,于一铭,崔若瑜",民生证券,80000038,通信设备,448,移远通信,603236,2022-01-29 00:00:00.000,业绩预告点评：营收利润高速增长，车载模组加速布局,移远通信(603236)事件：1月27日晚，移远通信发布2021年业绩预告，预计2021年全...,https://data.eastmoney.com/report/zw_stock.jsh...
497,吕伟,民生证券,80000038,软件开发,737,深信服,300454,2022-01-29 00:00:00.000,深信服2021年业绩预告点评：转型短期阵痛，不改长期XaaS高景气,深信服(300454)事件：1月25日，公司发布2021年年度业绩预告，2021年预计实现营...,https://data.eastmoney.com/report/zw_stock.jsh...
498,"周泰,曾杰煌",民生证券,80000038,化纤行业,471,桐昆股份,601233,2022-01-29 00:00:00.000,2021年年度业绩预增公告点评：21年业绩高增，产能扩张有望驱动长期盈利增长,桐昆股份(601233)事件： 2022 年 1 月 28 日，公司发布 2021 年年度业...,https://data.eastmoney.com/report/zw_stock.jsh...


In [2]:
import paddlenlp
from paddlenlp import Taskflow

senta = Taskflow("sentiment_analysis")

In [4]:
pattern = r';|\'|`|\?|\~|!|@|#|\^|&|=|\_|。|；|·|！|？|…'
sp_pattern = r'，|-|\{|\}|<|>|【|】|\+|,'
ssp_pattern = r'：|:|、'
def divide_sen(sentence):
    sentence = sentence.replace(" ", "")
    result_list = re.split(pattern, sentence)
    for sen in result_list:
        if len(sen) > 300:
            temp_list = re.split(sp_pattern, sen)
            for temp in temp_list:
                result_list.append(temp)
            result_list.remove(sen)
    for sen in result_list:
        if len(sen) > 300:
            temp_list = re.split(ssp_pattern, sen)
            for temp in temp_list:
                result_list.append(temp)
            result_list.remove(sen)
    return result_list

In [6]:
data_path = "个股研报\个股研报_"
output_path = "个股研报\Stock_"
begin_report = 1
begin_point = 0
total_report = 28

In [7]:
def research_report_ema(df_report):
    temp_res = []
    report_data = df_report.values
    sentence = report_data[:,9]
    sentence = sentence.tolist()
    print('start senta')
    for i in range(0,len(sentence)):
            ema = senta(sentence[i])
            temp_res.append({
            "researcher":report_data[i][0],
            "orgSName":report_data[i][1],
            "orgCode":report_data[i][2],
            "indvInduName":report_data[i][3],
            "indvInduCode":report_data[i][4],
            "stockName":report_data[i][5],
            "stockCode":report_data[i][6],
            "publishDate":report_data[i][7],
            "title":report_data[i][8],
            "content":report_data[i][9],
            'ema_label':ema[0]['label'],
            'ema_score':ema[0]['score']
        })
    print('finish senta')
    return temp_res

In [2]:
    research_report = json.load(open('个股研报\Stock_Allx','r',encoding='utf8'))
    df_report = pd.DataFrame.from_dict(research_report, orient='columns')

In [8]:
for num in range(begin_report, total_report + 1):
    full_data_path = data_path + str(num) + ".json"
    full_output_path = output_path + str(num) + ".json"
    research_report = json.load(open(full_data_path,'r',encoding='utf8'))
    df_report = pd.DataFrame.from_dict(research_report, orient='columns')
    print("Processing Report:", num)
    tmp=[]
    for i in range(0,len(df_report)):
        if df_report.iloc[i]['content']=='':
            tmp.append(i)
    df_report=df_report.drop(labels=tmp)
    df_report=df_report.reset_index(drop=True)
    temp_res = research_report_ema(df_report)
    with open(full_output_path, 'w',encoding='utf8') as f:
        json.dump(temp_res, f, indent=1, ensure_ascii = False)
    begin_point = 0
    

Processing Report: 1
start senta
finish senta
Processing Report: 2
start senta
finish senta
Processing Report: 3
start senta
finish senta
Processing Report: 4
start senta
finish senta
Processing Report: 5
start senta
finish senta
Processing Report: 6
start senta
finish senta
Processing Report: 7
start senta
finish senta
Processing Report: 8
start senta
finish senta
Processing Report: 9
start senta
finish senta
Processing Report: 10
start senta
finish senta
Processing Report: 11
start senta
finish senta
Processing Report: 12
start senta
finish senta
Processing Report: 13
start senta
finish senta
Processing Report: 14
start senta
finish senta
Processing Report: 15
start senta
finish senta
Processing Report: 16
start senta
finish senta
Processing Report: 17
start senta
finish senta
Processing Report: 18
start senta
finish senta
Processing Report: 19
start senta
finish senta
Processing Report: 20
start senta
finish senta
Processing Report: 21
start senta
finish senta
Processing Report: 22


In [40]:
data_path = "个股研报\个股研报_"
output_path = "个股研报\Stock_"
begin_report = 2
begin_point = 0
total_report = 28

research_report = json.load(open('个股研报\Stock_1.json','r',encoding='utf8'))
df_report_whole = pd.DataFrame.from_dict(research_report, orient='columns')

for num in range(begin_report, total_report + 1):
    full_data_path = output_path + str(num) + ".json"
    research_report = json.load(open(full_data_path,'r',encoding='utf8'))
    df_report = pd.DataFrame.from_dict(research_report, orient='columns')
    df_report_whole=df_report_whole.append(df_report,ignore_index=True)
    del df_report
    

In [49]:
report_data=df_report_whole.values
temp_res=[]
for i in range(0,len(df_report_whole)):
            temp_res.append({
            "researcher":report_data[i][0],
            "orgSName":report_data[i][1],
            "orgCode":report_data[i][2],
            "indvInduName":report_data[i][3],
            "indvInduCode":report_data[i][4],
            "stockName":report_data[i][5],
            "stockCode":report_data[i][6],
            "publishDate":report_data[i][7],
            "title":report_data[i][8],
            "content":report_data[i][9],
            'ema_label':report_data[i][10],
            'ema_score':report_data[i][11]
        })

with open("个股研报\Stock_Allx", 'w',encoding='utf8') as f:
    json.dump(temp_res, f, indent=1, ensure_ascii = False)

In [47]:
df_report_whole=df_report_whole.drop_duplicates(ignore_index=True)

In [5]:
research_report = json.load(open('个股研报\Stock_Allx','r',encoding='utf8'))
df_report = pd.DataFrame.from_dict(research_report, orient='columns')
df_report['ema_score']=df_report['ema_score'].astype(object)

In [9]:
df_report_test=df_report
fun={'stockName' : 'first', 'stockCode' : 'first', 'ema_label' : lambda x: x.append(x), 'ema_score' : lambda x: x.append(x), 'content' : lambda y: y.append(y), 'publishDate' : lambda z: z.append(z)}
df_report_test=df_report_test.groupby('stockCode').aggregate(fun)

In [10]:
df_report_test=df_report_test.reset_index(drop=True)

In [11]:
df_report_test

,stockName,stockCode,ema_label,ema_score,content,publishDate
0,平安银行,000001,"[negative, negative, positive, negative, negat...","[0.8886626958847046, 0.9691681861877441, 0.878...",[平安银行(000001)事件公司发布了2021年度业绩快报。营收增速扩大，净利增速受拨备多...,"[2022-01-18 00:00:00.000, 2022-01-18 00:00:00...."
1,万科A,000002,"[negative, negative, negative, negative, negat...","[0.9045731425285339, 0.8285372853279114, 0.924...",[万科A(000002)事件：公司发布 2022 年 2 月经营情况公告， 2 月单月实现销...,"[2022-03-06 00:00:00.000, 2022-02-23 00:00:00...."
2,南玻A,000012,"[negative, negative, negative, negative, negat...","[0.788520336151123, 0.9858942031860352, 0.9874...",[南玻A(000012)核心观点公司发布 21 年业绩预告，归母净利润 12.9-16.3 ...,"[2022-02-08 00:00:00.000, 2021-12-31 00:00:00...."
3,深科技,000021,"[negative, negative]","[0.9878907203674316, 0.9878907203674316]",[深科技(000021)一、事件概述2021/8/25 公司发布半年报， 实现营收 80 亿...,"[2021-08-26 00:00:00.000, 2021-08-26 00:00:00...."
4,飞亚达,000026,"[positive, negative, positive, negative, negat...","[0.6035376191139221, 0.7990429997444153, 0.874...",[飞亚达(000026)报告关键要素:公司披露 2021 年第三季度报告，报告显示，公司 2...,"[2021-10-26 00:00:00.000, 2021-10-26 00:00:00...."
...,...,...,...,...,...,...
1943,汉鑫科技,837092,"[positive, positive]","[0.9291233420372009, 0.9291233420372009]",[汉鑫科技(837092)主要观点：专注三大核心业务， 缔造区域领导品牌。公司是信息系统集成...,"[2021-11-26 00:00:00.000, 2021-11-26 00:00:00...."
1944,广道高新,839680,"[positive, positive]","[0.8243260979652405, 0.8243260979652405]",[广道高新(839680)主要观点：广道高新： 数据感知及分析的嵌入式软硬件设备提供商。公司...,"[2021-11-26 00:00:00.000, 2021-11-26 00:00:00...."
1945,永顺生物,839729,"[negative, positive, negative, positive]","[0.9607420563697815, 0.8417075276374817, 0.960...",[永顺生物(839729)主要观点：专注兽用生物制品，猪瘟疫苗龙头企业。公司成立于2002年...,"[2022-01-07 00:00:00.000, 2021-09-03 00:00:00...."
1946,晶赛科技,871981,"[positive, negative, positive, negative]","[0.9099153876304626, 0.9375831484794617, 0.909...",[晶赛科技(871981)主要观点：国内石英晶振供应商龙头，行业地位显著。公司深耕石英晶振行...,"[2022-02-21 00:00:00.000, 2021-11-15 00:00:00...."


In [12]:
df_report_test.to_csv('stock_result1.csv',index=False)


## 行业研报

In [15]:
research_report = json.load(open('company_ema.json','r',encoding='utf8'))
df_report = pd.DataFrame.from_dict(research_report, orient='columns')

In [28]:
tmp=[] #删掉非上市公司
for i in range(0,len(df_report)):
    if df_report.iloc[i]['stock']==[]:
        tmp.append(i)
df_report=df_report.drop(labels=tmp)
df_report=df_report.reset_index(drop=True)

In [50]:
tmp=[] #删掉境外公司
for i in range(0,len(df_report)):
    if len(df_report.iloc[i]['stock'][0])!=9:
        tmp.append(i)

In [52]:
df_report=df_report.drop(labels=tmp)
df_report=df_report.reset_index(drop=True)

In [79]:
df_report_test.to_csv('stock_result3.csv',index=False)

In [6]:
dataf=pd.read_csv('stock_result2.csv')

In [7]:
dataf

,company,stock,time,sentence,ema_label,ema_score
0,江苏银行股份有限公司,['600919.SH'],"['2022-03-07 00:00:00.000', '2022-03-07 00:00:...",['个股方面，建议关注业绩增速较快的城农商行：如江苏银行、常熟银行、杭州银行等，核心标的持续...,"['positive', 'positive', 'negative', 'positive...","[0.7369949817657471, 0.5980978012084961, 0.550..."
1,招商银行股份有限公司,['600036.SH'],"['2022-03-07 00:00:00.000', '2022-03-07 00:00:...",['个股方面，建议关注业绩增速较快的城农商行：如江苏银行、常熟银行、杭州银行等，核心标的持续...,"['positive', 'positive', 'positive', 'positive...","[0.7369949817657471, 0.7878919243812561, 0.677..."
2,宁波银行股份有限公司,['002142.SZ'],"['2022-03-07 00:00:00.000', '2022-03-07 00:00:...",['个股方面，建议关注业绩增速较快的城农商行：如江苏银行、常熟银行、杭州银行等，核心标的持续...,"['positive', 'positive', 'positive', 'positive...","[0.7369949817657471, 0.7878919243812561, 0.736..."
3,中国邮政储蓄银行股份有限公司,['601658.SH'],"['2022-03-07 00:00:00.000', '2022-03-07 00:00:...",['个股方面，建议关注业绩增速较快的城农商行：如江苏银行、常熟银行、杭州银行等，核心标的持续...,"['positive', 'negative', 'positive', 'positive...","[0.7369949817657471, 0.5505151152610779, 0.787..."
4,杭州银行股份有限公司,['600926.SH'],"['2022-03-07 00:00:00.000', '2022-03-07 00:00:...",['个股方面，建议关注业绩增速较快的城农商行：如江苏银行、常熟银行、杭州银行等，核心标的持续...,"['positive', 'negative', 'positive', 'positive...","[0.7369949817657471, 0.5505151152610779, 0.787..."
...,...,...,...,...,...,...
4606,河北省公安厅服务中心保安服装经销部,['002822.SZ'],['2021-09-02 00:00:00.000'],['（云酒头条）2、市场监管总局办公厅联合教育部办公厅、国家卫生健康委办公厅、公安部办公厅印...,['negative'],[0.668020486831665]
4607,福鞍控股有限公司,['603315.SH'],['2021-09-02 00:00:00.000'],['（电动知家）公司动态贝特瑞（835185）：公司拟与福鞍控股有限公司签署《关于年产10万...,['positive'],[0.8361352682113647]
4608,上海振华重工（集团）股份有限公司,['600320.SH'],['2021-09-02 00:00:00.000'],['此外，海上风电的施工还涉及安装船、起重工程船等，建造单位有振华重工、中远海运重工'],['positive'],[0.6287190318107605]
4609,山东海化集团有限公司,['000822.SZ'],['2021-09-02 00:00:00.000'],['山东海化：公司主要从事纯碱、溴素、氯化钙等产品的生产和销售'],['positive'],[0.8976718783378601]
